---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [2]:
def date_sorter():
    
    # Your code here
    global df
    import re
    import datetime
    import numpy as np
    exp_str_list = [r'(?P<original>(?P<month>\d{,2}[/-])(?P<date>\d?\d[/-])(?P<year>\d{2,4}))',\
                    r'(?P<original>(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-.\s]*)(?P<date>(?:\d{1,2})[?: -.,]*)(?P<year>\d{2,4}))',\
                    r'(?P<original>(?P<date>(?:\d{1,2})[\s]*)(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[?: -.]*)(?P<year>\d{2,4}))',\
                    r'(?P<original>(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[?: .]*)(?P<date>(?:\d{1,2})[a-z]*[?: ,]*)(?P<year>\d{2,4}))',\
                    r'(?P<original>(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[\s]*)(?P<year>\d{2,4}))'] 

    #Defining a function
    def get_extracts(extracts,df,df_index,exp_str):
        extract_intermediate = df[df_index].str.extractall(exp_str)
        extracts = pd.concat([extracts,extract_intermediate])
        extracts = extracts.iloc[extracts.index.labels[1] == 0]

        extracts_index = [i for i in extracts.index.levels[0]]
        index_remaining = df.index[~df.index.isin(extracts_index)]

        return (extracts, index_remaining)

    #Calling function
    extracts  = pd.DataFrame()
    index_remaining = [i for i in df.index]
    #Getting extracts for a few rules
    for exp_str in exp_str_list:
        extracts, index_remaining = get_extracts(extracts,df,index_remaining,exp_str)

    #Dealing with dates without day number or day number and month:
    #Without day
    #1
    extract_without_day = df[index_remaining].str.extractall(r'(?P<original>(?P<month>(?:(\d{1,2})/))(?P<year>\d{2,4}))') #Without date
    extract_without_day['date'] = 1
    extracts = pd.concat([extracts,extract_without_day])

    extracts_index = [i for i in extracts.index.levels[0]]
    index_remaining = df.index[~df.index.isin(extracts_index)]

    #2
    extract_without_day2 = df[index_remaining].str.extractall(r'(?P<original>(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-.\s,]*)(?P<year>\d{2,4}))')
    extract_without_day2['date'] = 1
    extracts = pd.concat([extracts,extract_without_day2])

    extracts_index = [i for i in extracts.index.levels[0]]
    index_remaining = df.index[~df.index.isin(extracts_index)]

    #Without month
    extract_without_day_month = df[index_remaining].str.extractall(r'(?P<original>(?P<year>\d{4}))') #Without date
    extract_without_day_month['date'] = 1
    extract_without_day_month['month'] = 1
    extracts = pd.concat([extracts,extract_without_day_month])
    extracts_index = [i for i in extracts.index.levels[0]]

    extracts = extracts.iloc[extracts.index.labels[1] == 0]

    #Cleaning extracted dates
    #Changing year
    extracts['year'] = extracts['year'].astype(str)  
    arr = list()
    for i in extracts['year']:
        if (len(i) == 2):
            if (i[0] != '1'):
                if (i[0] != '0'):
                    a = '19' + i
                else:
                    a = '20' + i
            else:
                a = '20' + i
        else:
            a = i
        arr.append(a)
    extracts['year'] = arr
    
#     #Changing month
    month_dict = {'January': 1, 'Jan': 1,'February':2,'Feb': 2,'March': 3,'Mar':3,'April':4,"Apr":4,'May':5,'Jun':6,'June':6,
                  'July':7,'Jul':7,'August':8,'Aug':8,'September':9,'Sep':9,'October':10,'Oct':10,'November':11,'Nov':11,
                  'December':12,'Dec':12,'Janaury':1,'Decemeber':12 }
    extracts['month'] = extracts['month'].astype(str)
    extracts['month'] = extracts['month'].str.extract(r'(\w{,10})')
    extracts['month'] = extracts['month'].apply(lambda x: month_dict[x] if x in month_dict.keys() else x)
    extracts['month'] = extracts['month'].astype(str)
    extracts['month'] = extracts['month'].apply(lambda x: '0'+x if len(x)==1 else x)


    #Changing days
    extracts['date'] = extracts['date'].astype(str)
    extracts['date'] = extracts['date'].str.findall(r'(\d{1,2})').apply(lambda x: x[0])
    extracts['date'] = extracts['date'].apply(lambda x: '0'+x if len(x)==1 else x)
    
    #Handling some anomaly
    extracts['date'] = np.where(extracts['month'] == '', '01',extracts['date'])
    extracts['month'] = np.where(extracts['month'] == '', '08',extracts['month'])
    
    #Creating a new cleaned date column
    extracts['new_date'] = extracts['month'] + '-' + extracts['date'] + '-' + extracts['year']
    #Convert to datetime
    extracts['new_date'] = extracts['new_date'].apply(lambda x: datetime.datetime.strptime(x,'%m-%d-%Y'))
    extracts.sort_values(by='new_date', inplace=True)
    sorted_date_indices = pd.Series(extracts.index.labels[0])
    return sorted_date_indices